In [1]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression    
from sklearn.naive_bayes import GaussianNB          
from sklearn.ensemble import RandomForestClassifier    

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import KFold

In [2]:
import os
import torch

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
torch.cuda.is_available()

True

In [5]:
# 임베딩 배열을 데이터프레임으로 변환
def make_df(data):
    
    array = data[0]     # 첫번째 array
    error = []          # 에러 인덱스 리스트
    

    for index in range(len(data)-1):
        plus_array = data[index+1]     # 추가할 array
        try:
            array = np.concatenate((array, plus_array), axis=0) 
        except:
            print(index+1, '번 인덱스 오류 처리 완료')
            error.append(index+1)     # 에러 인덱스 저장
            
        # 201건부터는 모두 보이스피싱 텍스트 임베딩값
        # 보이스피싱 여부를 붙이고자 phishing_check로 인덱스 지정
        if (index+1) == 199:
            phishing_check = array.shape[0] + 1

    
    # 데이터프레임 변환 및 레이블 지정
    df = pd.DataFrame(array)
    df['phishing'] = 0                        # 상담 텍스트 임베딩값
    df.loc[phishing_check:,'phishing'] = 1    # 보이스파싱 텍스트 임베딩값
    return df

In [17]:
def data_split(df):
    feature = df.iloc[:,:-1]      # 피처 칼럼: 예측에 사용되는 데이터
    target  = df['phishing']      # 타겟 칼럼: 예측(분류) 대상 데이터
    x_train, x_valid, y_train, y_valid = train_test_split(feature, target, test_size=0.2,
                                                          shuffle=True,      
                                                          stratify=target,  
                                                          random_state=1234) 
    return x_train, x_valid, y_train, y_valid, feature, target

In [7]:
eng_ratio5_4gram = np.load('eng_ratio5_4gram_embedding.npy', allow_pickle=True)

In [16]:
type(eng_ratio5_4gram)

numpy.ndarray

In [14]:
import pickle


df = make_df(eng_ratio5_4gram)
x_train, x_valid, y_train, y_valid, feature, target = data_split(df)

reg = LogisticRegression(random_state=0, max_iter=500)
reg.fit(x_train, y_train)
pred = reg.predict(x_valid) 

saved_model = pickle.dumps(reg)

with open('saved_model_file', 'wb') as f:
    pickle.dump(reg, f)
    
    
clf_from_pickle = pickle.loads(saved_model)
clf_from_pickle.predict(x_valid)

264 번 인덱스 오류 처리 완료
291 번 인덱스 오류 처리 완료
304 번 인덱스 오류 처리 완료
316 번 인덱스 오류 처리 완료
330 번 인덱스 오류 처리 완료
331 번 인덱스 오류 처리 완료
332 번 인덱스 오류 처리 완료
334 번 인덱스 오류 처리 완료


array([1, 0, 1, ..., 1, 0, 0], dtype=int64)

In [18]:
type(x_valid)

pandas.core.frame.DataFrame